### Обработка данных 

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from random import randint
import re

In [2]:
def caesar_cipher(text: str, k: int) -> str:
    encrypted_text = ""
    for s in text:
        order = new_order = ord(s)
        if s.isalpha():
            new_order = order + k
            if s.isupper():
                if new_order > ord("Я"):
                    new_order = ord('А') + order + k - ord('Я') - 1
            else:
                if new_order > ord("я"):
                    new_order = ord('а') + order + k - ord('я') - 1
        encrypted_text += chr(new_order)
    return encrypted_text

In [3]:
CHARS = list('абвгдежзийклмнопрстуфхцчшщъыьэюя ')
INDEX_TO_CHAR = ['none'] + [w for w in CHARS]
CHAR_TO_INDEX = {w: i for i, w in enumerate(INDEX_TO_CHAR)}

In [4]:
# все строки списками из индексов
def raw_to_indexes(lines: list[str]):
    indexes = torch.zeros((len(lines), MAX_LEN), dtype=int)
    for i in range(len(lines)):
        if raw_lines[i]:
            for j, w in enumerate(lines[i]):
                if j >= MAX_LEN: break
                indexes[i, j] = CHAR_TO_INDEX.get(w, CHAR_TO_INDEX['none'])
    return indexes

In [5]:
# обучающие данные
MAX_LEN = 20

# список всех строк
with open('datasets/onegin.txt', 'r', encoding='utf-8') as file:
    raw_lines = [re.sub(r'[^а-яА-Я ]*', '', line.strip().lower()) for line in file.readlines()]
    
# список всех строк (зашифрованный)
encrypted_raw_lines = [caesar_cipher(i, randint(1,10)) for i in raw_lines]

# все строки списками из индексов
indexes = raw_to_indexes(raw_lines)

# все зашифрованные строки списками из индексов
encrypted_indexes = raw_to_indexes(encrypted_raw_lines)

### Полученные тензоры данных

In [6]:
raw_lines[0:3]

['мой дядя самых честных правил',
 'когда не в шутку занемог',
 'он уважать себя заставил']

In [7]:
encrypted_raw_lines[0:3]

['орл жбжб увоэч щзуфпэч ствдкн',
 'пуийе тк з эшчпш метксуи',
 'цх ыкиоиъд щнйз пищъикру']

In [8]:
indexes[0:3]

tensor([[13, 15, 10, 33,  5, 32,  5, 32, 33, 18,  1, 13, 28, 22, 33, 24,  6, 18,
         19, 14],
        [11, 15,  4,  5,  1, 33, 14,  6, 33,  3, 33, 25, 20, 19, 11, 20, 33,  8,
          1, 14],
        [15, 14, 33, 20,  3,  1,  7,  1, 19, 29, 33, 18,  6,  2, 32, 33,  8,  1,
         18, 19]])

In [9]:
encrypted_indexes[0:3]

tensor([[15, 17, 12, 33,  7,  2,  7,  2, 33, 20,  3, 15, 30, 24, 33, 26,  8, 20,
         21, 16],
        [16, 20,  9, 10,  6, 33, 19, 11, 33,  8, 33, 30, 25, 24, 16, 25, 33, 13,
          6, 19],
        [23, 22, 33, 28, 11,  9, 15,  9, 27,  5, 33, 26, 14, 10,  8, 33, 16,  9,
         26, 27]])

In [10]:
indexes.shape, encrypted_indexes.shape # 1452 строки, длиной MAX_LEN символов

(torch.Size([1452, 20]), torch.Size([1452, 20]))

### Создание и обучение модели 

In [11]:
class Decryptor(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Decryptor, self).__init__()
        self.embedding = nn.Embedding(input_size, 64)
        self.rnn = nn.GRU(64, hidden_size, batch_first=True)
        self.linear = nn.Linear(hidden_size, output_size)

    def forward(self, input):
        embedded = self.embedding(input)
        output, _ = self.rnn(embedded)
        output = self.linear(output)
        return output

In [12]:
input_size = len(CHAR_TO_INDEX)
hidden_size = 256
output_size = len(CHAR_TO_INDEX)

In [13]:
model = Decryptor(input_size, hidden_size, output_size)

In [14]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [15]:
num_epochs = 200
for epoch in range(num_epochs):
    
    for i in range(int(len(indexes) /100)):
        X_batch = encrypted_indexes[i * 100 : (i + 1) * 100] # 100 x 20
        Y_batch = indexes[i * 100 : (i + 1) * 100]           # 100 x 20
        Y_batch = Y_batch.flatten()                          # 2000
        
        optimizer.zero_grad()
        
        output = model(X_batch)                              # 100 x 20 x 64
        output = output.view(-1, output_size)                # 2000 x 64
        
        loss = criterion(output, Y_batch)
        loss.backward()
        optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f'Epoch: {epoch + 1}, Loss: {loss.item()}')

Epoch: 10, Loss: 1.3200126886367798
Epoch: 20, Loss: 0.9491628408432007
Epoch: 30, Loss: 0.6332158446311951
Epoch: 40, Loss: 0.37963682413101196
Epoch: 50, Loss: 0.2857684791088104
Epoch: 60, Loss: 0.26886847615242004
Epoch: 70, Loss: 0.28517383337020874
Epoch: 80, Loss: 0.16235783696174622
Epoch: 90, Loss: 0.1457023322582245
Epoch: 100, Loss: 0.13592620193958282
Epoch: 110, Loss: 0.12946967780590057
Epoch: 120, Loss: 0.12498925626277924
Epoch: 130, Loss: 0.12177874147891998
Epoch: 140, Loss: 0.11942095309495926
Epoch: 150, Loss: 0.11765575408935547
Epoch: 160, Loss: 0.11631161719560623
Epoch: 170, Loss: 0.11527378857135773
Epoch: 180, Loss: 0.11446241289377213
Epoch: 190, Loss: 0.11382022500038147
Epoch: 200, Loss: 0.11330588161945343


### Тестирование данных (обучающая выборка)

In [16]:
with torch.no_grad():
    output = model(encrypted_indexes)
    predicted_indexes = torch.argmax(output, dim=2)
    decrypted_indexes = predicted_indexes.numpy()

# Расшифрованные строки
decrypted_strings = []
for decrypted_index in decrypted_indexes:
    decrypted_string = ''.join([INDEX_TO_CHAR[i] for i in decrypted_index if i != 0])
    decrypted_strings.append(decrypted_string)

In [17]:
decrypted_strings[0:20]

['иой дядя самых честн',
 'иогда не в шутку зан',
 'но уважать себя заст',
 'и лучше выдумать не ',
 'иво пример другим на',
 'пр боже мой какая ск',
 'н больным сидеть и д',
 'ие отходя ни шагу пр',
 'иакое низкое коварст',
 'нолуживого забавлять',
 'иму подушки поправля',
 'иачально подносить л',
 'дздыхать и думать пр',
 'когда же черт возьме',
 '',
 'пак думал молодой по',
 'иетя в пыли на почто',
 'дсевышней волею зеве',
 'паследник всех своих',
 'воузья людмилы и рус']

### Тестирование данных (тестовая выборка - onegin_test)

In [19]:
# список всех строк
with open('datasets/onegin_test.txt', 'r', encoding='utf-8') as file:
    raw_lines = [re.sub(r'[^а-яА-Я ]*', '', line.strip().lower()) for line in file.readlines()]
    
# список всех строк (зашифрованный)
encrypted_raw_lines = [caesar_cipher(i, randint(1,10)) for i in raw_lines]

# все строки списками из индексов
indexes = raw_to_indexes(raw_lines)

# все зашифрованные строки списками из индексов
encrypted_indexes = raw_to_indexes(encrypted_raw_lines)

with torch.no_grad():
    output = model(encrypted_indexes)
    predicted_indexes = torch.argmax(output, dim=2)
    decrypted_indexes = predicted_indexes.numpy()

# Расшифрованные строки
decrypted_strings = []
for decrypted_index in decrypted_indexes:
    decrypted_string = ''.join([INDEX_TO_CHAR[i] for i in decrypted_index if i != 0])
    decrypted_strings.append(decrypted_string)
    
decrypted_strings[20:40]

['нт что ж ты ддечь оч',
 'вх тлучай ленский да',
 'чвздеть мне узклиду ',
 'предмет и мыслей и п',
 'н слеж и рифм  ',
 'представь меня  ты ч',
 ' б сад  лрежа зе  уо',
 'нои с охотой примут ',
 'инедем  поскакали др',
 'велолца мр уеххтыйря',
 'норой сяжелые услуви',
 'ирутерриимоой старин',
 'ибрад известный угоы',
 'песут на блюдечках в',
 'иа столик ставят вош',
 'инвчин р брурмичною ',
 'нол доуперк уворл мт',
 'волми кетят во весь ',
 'чжперь подркучаем тк',
 'иороев наших разгово']